# Many-to-One RNN for Football Match Outcome Prediction
A simple many-to-one RNN setup. Each match is a sequence of events (passes, shots, etc.), and the model predicts the **final match outcome**: home win / draw / away win.


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 1. Mock dataset generation
We create synthetic event sequences to mimic real football match data.

In [15]:
# Dataset parameters
batch_size = 32
seq_len = 100           # number of events per match
num_players = 500
num_teams = 20
num_event_types = 8
num_classes = 3         # home win, draw, away win

# Generate random sequences
player_from = torch.randint(0, num_players, (batch_size, seq_len))
player_to   = torch.randint(0, num_players, (batch_size, seq_len))
team        = torch.randint(0, num_teams, (batch_size, seq_len))
event_type  = torch.randint(0, num_event_types, (batch_size, seq_len))

# Labels for each match (one label per sequence)
labels = torch.randint(0, num_classes, (batch_size,))

### 2. Many-to-One RNN Model Definition

In [16]:
class ManyToOneRNN(nn.Module):
    def __init__(self, num_players, num_teams, num_event_types,
                 embed_dim=64, hidden_dim=128, num_layers=1, num_classes=3):
        super(ManyToOneRNN, self).__init__()
        
        # Embeddings for categorical inputs
        self.player_embed = nn.Embedding(num_players, embed_dim)
        self.team_embed   = nn.Embedding(num_teams, embed_dim // 2)
        self.event_embed  = nn.Embedding(num_event_types, embed_dim // 2)
        
        # Vanilla RNN
        input_dim = embed_dim*2 + embed_dim//2 + embed_dim//2
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Classifier head
        self.fc1 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc2 = nn.Linear(hidden_dim // 2, num_classes)
    
    def forward(self, player_from, player_to, team, event_type):
        # Embed categorical features
        from_emb = self.player_embed(player_from)   # [B, L, D]
        to_emb   = self.player_embed(player_to)
        team_emb = self.team_embed(team)
        event_emb= self.event_embed(event_type)
        
        # Concatenate embeddings
        x = torch.cat([from_emb, to_emb, team_emb, event_emb], dim=-1)  # [B, L, input_dim]
        
        # RNN forward
        _, h = self.rnn(x)  # h: [num_layers, B, hidden_dim]
        h = h[-1]           # Take last layer hidden state
        
        # Classifier
        z = F.relu(self.fc1(h))
        out = self.fc2(z)   # [B, num_classes]
        return F.log_softmax(out, dim=-1)

### 3. Model Initialization

In [17]:
model = ManyToOneRNN(num_players, num_teams, num_event_types, 
                     embed_dim=64, hidden_dim=128, num_layers=1, num_classes=num_classes)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 4. Forward pass 

In [18]:
outputs = model(player_from, player_to, team, event_type)
print("Output shape:", outputs.shape)  # [batch_size, num_classes]

loss = criterion(outputs, labels)
print("Initial loss:", loss.item())

Output shape: torch.Size([32, 3])
Initial loss: 1.1229115724563599


### 5. Simple Training Loop

In [19]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(player_from, player_to, team, event_type)
    
    # Compute loss
    loss = criterion(outputs, labels)
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    # Accuracy
    preds = outputs.argmax(dim=1)
    acc = (preds == labels).float().mean().item()
    
    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f} | Accuracy: {acc:.4f}")

Epoch 1 | Loss: 1.1229 | Accuracy: 0.3750
Epoch 2 | Loss: 1.0373 | Accuracy: 0.6250
Epoch 3 | Loss: 0.9624 | Accuracy: 0.9375
Epoch 4 | Loss: 0.8949 | Accuracy: 1.0000
Epoch 5 | Loss: 0.8317 | Accuracy: 1.0000
Epoch 6 | Loss: 0.7720 | Accuracy: 1.0000
Epoch 7 | Loss: 0.7147 | Accuracy: 1.0000
Epoch 8 | Loss: 0.6585 | Accuracy: 1.0000
Epoch 9 | Loss: 0.6031 | Accuracy: 1.0000
Epoch 10 | Loss: 0.5479 | Accuracy: 1.0000
Epoch 11 | Loss: 0.4941 | Accuracy: 1.0000
Epoch 12 | Loss: 0.4420 | Accuracy: 1.0000
Epoch 13 | Loss: 0.3916 | Accuracy: 1.0000
Epoch 14 | Loss: 0.3431 | Accuracy: 1.0000
Epoch 15 | Loss: 0.2969 | Accuracy: 1.0000
Epoch 16 | Loss: 0.2533 | Accuracy: 1.0000
Epoch 17 | Loss: 0.2131 | Accuracy: 1.0000
Epoch 18 | Loss: 0.1767 | Accuracy: 1.0000
Epoch 19 | Loss: 0.1441 | Accuracy: 1.0000
Epoch 20 | Loss: 0.1159 | Accuracy: 1.0000
